## code for concatenate tx and rx data with same size

In [2]:
import os
import numpy as np

# Function to read data from a file and convert it into a numpy array
def read_file_to_array(file_path, dtype):
    with open(file_path, 'r') as f:
        data = f.read().strip().split(',')
        return np.array([dtype(i) for i in data])

# Directories containing tx and rx data
tx_directory = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\5_test_TX_message_I128'
rx_directory = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\7_test_rx_decoded_data'

# Output paths for merged files
output_tx_file = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_tx_data.npy'
output_rx_file = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_rx_noisy_data.npy'

# Initialize lists to hold all tx and rx data
all_tx_data = []
all_rx_data = []

# Iterate through all tx files and corresponding rx subfolders and files
for file_name in sorted(os.listdir(tx_directory)):
    if file_name.endswith('.txt'):  # Ensure we're processing .txt files in the tx folder
        tx_file_path = os.path.join(tx_directory, file_name)
        
        # Find corresponding folder in the rx_directory with the same name as tx file (without extension)
        corresponding_rx_folder = os.path.join(rx_directory, file_name.split('.')[0])
        
        if os.path.exists(corresponding_rx_folder):  # Ensure the corresponding rx folder exists
            # Count the number of .txt files in the corresponding rx folder
            num_rx_files = len([f for f in os.listdir(corresponding_rx_folder) if f.endswith('.txt')])
            # Print the folder name and number of txt files inside
            #print(f"Folder: {corresponding_rx_folder}, Number of .txt files: {num_rx_files}")

            # Iterate through all .txt files in the corresponding rx folder
            for rx_file_name in sorted(os.listdir(corresponding_rx_folder)):
                if rx_file_name.endswith('.txt'):
                    rx_file_path = os.path.join(corresponding_rx_folder, rx_file_name)
                    
                    # Read tx and rx data
                    tx_data = read_file_to_array(tx_file_path, int)
                    rx_data = read_file_to_array(rx_file_path, float)
                    
                    # Append the tx and rx data to lists
                    all_tx_data.append(tx_data)
                    all_rx_data.append(rx_data)
                    
                    # Print progress for each file
                    #print(f"Processed {len(tx_data)}/{num_rx_files}")

# Concatenate all tx and rx data into single arrays
concatenated_tx_data = np.concatenate(all_tx_data)
concatenated_rx_data = np.concatenate(all_rx_data)

# Ensure both TX and RX data have the same length before reshaping
min_length = min(len(concatenated_tx_data), len(concatenated_rx_data))
concatenated_tx_data = concatenated_tx_data[:min_length]
concatenated_rx_data = concatenated_rx_data[:min_length]

# Reshape the concatenated data into (n, 16) where n is the largest multiple of 16
reshaped_tx_data = concatenated_tx_data[:min_length].reshape(-1,8, 8)
reshaped_rx_data = concatenated_rx_data[:min_length].reshape(-1,8, 8)
\


# Save the reshaped tx and rx data to files
np.save(output_tx_file, reshaped_tx_data)
np.save(output_rx_file, reshaped_rx_data)

# Print output paths and shapes
print(f"TX data shape: {reshaped_tx_data.shape}")
print(f"RX data shape: {reshaped_rx_data.shape}")
print(f"All tx data merged and saved to: {output_tx_file}")
print(f"All rx data merged and saved to: {output_rx_file}")


TX data shape: (3756, 8, 8)
RX data shape: (3756, 8, 8)
All tx data merged and saved to: F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_tx_data.npy
All rx data merged and saved to: F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_rx_noisy_data.npy


## clipped extreme data

In [5]:
import numpy as np


# Load the data from the text files
rx_noisy_data = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_rx_noisy_data.npy'
output_filee =  r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\test_clip_rx_noisy_data.npy'

rx_train = np.load(rx_noisy_data)



# Reshape data to (-1, 128)
rx_train_reshaped = rx_train.reshape(-1, 128)

# Clip extreme values in each batch
clipped_data = []
for batch in rx_train_reshaped:
    lower_threshold = np.percentile(batch, 5)
    upper_threshold = np.percentile(batch, 95)
    batch_clipped = np.clip(batch, lower_threshold, upper_threshold)
    clipped_data.append(batch_clipped)

clipped_data = np.array(clipped_data).reshape(-1)  # Flatten back to original shape

# Load the datasets into numpy arrays (assuming they're comma-separated)
clip_rx_noisy_data = clipped_data.reshape(-1,8,8)
np.save(output_filee, clip_rx_noisy_data)


In [2]:
import numpy as np

In [8]:
rx_noisy_data = r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\train_tx_data.npy'
clip_rx_noisy_data =  r'F:\My File\OCC_Nov_11\Datasets_L8_8_subcarrier\Datasets\train_tx_data.npy'

rx_train = np.load(rx_noisy_data)
clip_data = np.load(clip_rx_noisy_data)

print((rx_train.reshape(-1,128)).shape)
print(clip_data.shape)

(19090, 128)
(38180, 8, 8)


## complex dataset

In [7]:
import os
import numpy as np

# Function to read data from a .txt file and convert it into a numpy array
def read_file_to_array(file_path, dtype):
    with open(file_path, 'r') as f:
        data = f.read().strip().split(',')
        return np.array([dtype(i) for i in data])

# Directories containing tx and rx data
tx_directory = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\fft1_tx_message'
rx_directory = r'E:\Minhaz\iot\OCC_sep_5\New_dataset_28_10\rx_decoded_data1px'

# Output paths for merged files
output_tx_file = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\fft_merged_tx4.npy'
output_rx_file = r'E:\Minhaz\iot\OCC_sep_5\new_dataset_16_10\fft_merged_rx4.txt'

# Initialize lists to hold all tx and rx data
all_tx_data = []
all_rx_data = []
len_tx_data = 0
len_rx_data = 0

# Iterate through all tx .npy files and corresponding rx subfolders and files
for file_name in sorted(os.listdir(tx_directory)):
    if file_name.endswith('.npy'):  # Ensure we're processing .npy files in the tx folder
        tx_file_path = os.path.join(tx_directory, file_name)
        
        # Find corresponding folder in the rx_directory with the same name as tx file (without extension)
        corresponding_rx_folder = os.path.join(rx_directory, file_name.split('.')[0])
        
        if os.path.exists(corresponding_rx_folder):  # Ensure the corresponding rx folder exists
            # Count the number of .txt files in the corresponding rx folder
            num_rx_files = len([f for f in os.listdir(corresponding_rx_folder) if f.endswith('.txt')])
            # Print the folder name and number of txt files inside
            #print(f"Folder: {corresponding_rx_folder}, Number of .txt files: {num_rx_files}")

            # Iterate through all .txt files in the corresponding rx folder
            for rx_file_name in sorted(os.listdir(corresponding_rx_folder)):
                if rx_file_name.endswith('.txt'):
                    rx_file_path = os.path.join(corresponding_rx_folder, rx_file_name)
                    
                    # Read tx data from .npy and rx data from .txt
                    tx_data = np.load(tx_file_path)  # Load .npy file
                    rx_data = read_file_to_array(rx_file_path, float)  # Load .txt file
                    #print(tx_data)
                    # Append the tx and rx data to lists
                    all_tx_data.append(tx_data)
                    all_rx_data.append(rx_data)
                    len_tx_data += len(tx_data)
                    len_rx_data += len(rx_data)
                    
                    #print(all_tx_data)
                    # Print progress for each file
                    #print(f"Processed  {len_tx_data} = {len_rx_data}:{len(tx_data)} = {len(rx_data)}/{num_rx_files}")

            #print(all_tx_data)

# Concatenate all tx and rx data into single arrays
concatenated_tx_data = np.concatenate(all_tx_data, axis=0)
concatenated_rx_data = np.concatenate(all_rx_data)
#print(concatenated_tx_data[1])

# # Ensure both TX and RX data have the same length before reshaping
# min_length = min(len(concatenated_tx_data), len(concatenated_rx_data))
# concatenated_tx_data = concatenated_tx_data[:min_length]
# concatenated_rx_data = concatenated_rx_data[:min_length]

# Reshape the concatenated data into (n, 16) where n is the largest multiple of 16
#reshaped_tx_data = concatenated_tx_data.reshape(-1, 16)
#print(reshaped_tx_data.shape)
reshaped_rx_data = concatenated_rx_data.reshape(-1, 16)

# Save the reshaped tx data to a .npy file and rx data to a .txt file
np.save(output_tx_file, concatenated_tx_data)
np.savetxt(output_rx_file, reshaped_rx_data, fmt='%.4f', delimiter=',')

# Print output paths and shapes
print(f"TX data shape: {concatenated_tx_data.shape}")
print(f"TX data shape: {concatenated_tx_data}")
print(f"RX data shape: {reshaped_rx_data.shape}")
print(f"All tx data merged and saved to: {output_tx_file}")
print(f"All rx data merged and saved to: {output_rx_file}")


TX data shape: (13688, 16)
TX data shape: [[3357.         +0.j           15.63139359-21.24451952j
   -15.58578644+18.58578644j ...  -20.74303492+18.82542712j
   -15.58578644-18.58578644j   15.63139359+21.24451952j]
 [3347.         +0.j          -13.01327425+17.71987428j
    13.58578644-18.38477631j ...  -12.71987428+21.78571562j
    13.58578644+18.38477631j  -13.01327425-17.71987428j]
 [3348.         +0.j          -16.4737145 +19.86990288j
    15.55634919+13.17157288j ...   25.31856124+17.07695404j
    15.55634919-13.17157288j  -16.4737145 -19.86990288j]
 ...
 [3357.         +0.j          -21.25540818+18.25018172j
   -18.38477631-16.41421356j ...  -18.25018172-13.98333025j
   -18.38477631+16.41421356j  -21.25540818-18.25018172j]
 [3355.         +0.j           12.02081528-12.02081528j
   -19.79898987+16.79898987j ...  -12.02081528+12.02081528j
   -19.79898987-16.79898987j   12.02081528+12.02081528j]
 [3352.         +0.j          -14.18648347+18.80945224j
    15.55634919-15.55634919j ...